In [1]:
import sys
import os
# add parent directory (where "natsume" lives) to sys.path so I can import natsume
sys.path.append(r"C:\Users\WBS\Desktop\EXOPLANET WORK\006 NATSUME")
import natsume

import ttv_curvefit.curvefit as ttvfit  # custom-written
import ttvfast
from ttvfast import models

from tqdm import tqdm
import pandas as pd
import numpy as np
from astropy import units as u
from astropy.constants import M_earth
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
import seaborn as sns

# Import CSV for TTVFast simulations
df = pd.read_csv(r"C:\Users\WBS\Desktop\EXOPLANET WORK\006 NATSUME\validation\2nd_order_TTVFast\deckagol_TTVFast_params.csv")
df.head()

,e1,w1,e2,w2,Delta,mmr,j,N,P1,P2,m1,m2,mu1,mu2,validity1,validity2,valid
0,0.055991,58.822607,0.011141,50.888669,0.009918,7:5,7,2,7,8.247662,28.078984,0.545483,0.000084,0.000002,20.830953,5388.995909,False
1,0.087637,344.470331,0.035670,211.719067,0.007174,5:3,5,2,7,8.812771,79.232159,2574.401472,0.000238,0.007732,2.467673,0.186594,False
2,0.010202,48.065318,0.062583,220.054708,0.005091,5:1,5,4,7,8.794550,0.406798,25.491641,0.000001,0.000077,2079.712544,5.409987,False
3,0.031152,19.671023,0.027803,238.240708,0.005387,3:1,3,2,7,10.556567,156.776308,0.681225,0.000471,0.000002,1.978549,510.191044,False
4,0.086873,100.393352,0.031076,68.805124,0.008759,4:1,4,3,7,9.415083,129.163089,0.523876,0.000388,0.000002,2.276392,1568.974627,False


In [2]:
# Copy dataframe for fitting results
df_fit = df.copy()
df_fit_cols = ["A1", "B1", "V1", "Pttv1", "phase1", "std1", "A2", "B2", "V2", "Pttv2", "phase2", "std2", "R2_1", "R2_2"]
df_fit[df_fit_cols] = np.nan
df_fit.head()

,e1,w1,e2,w2,Delta,mmr,j,N,P1,P2,...,phase1,std1,A2,B2,V2,Pttv2,phase2,std2,R2_1,R2_2
0,0.055991,58.822607,0.011141,50.888669,0.009918,7:5,7,2,7,8.247662,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.087637,344.470331,0.035670,211.719067,0.007174,5:3,5,2,7,8.812771,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.010202,48.065318,0.062583,220.054708,0.005091,5:1,5,4,7,8.794550,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.031152,19.671023,0.027803,238.240708,0.005387,3:1,3,2,7,10.556567,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.086873,100.393352,0.031076,68.805124,0.008759,4:1,4,3,7,9.415083,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# For storing TTVs
df_ttv_cols = ["t0", "ttv0", "t1", "ttv1"]
df_ttv = pd.DataFrame(index=range(len(df)), columns=df_ttv_cols, dtype=object)
df_ttv.head()

,t0,ttv0,t1,ttv1
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN


In [4]:
# Create TTV simulations and extract Amplitude and Period
gravity = 0.000295994511 # AU^3/day^2/M_sun
stellar_mass = 1.0       # M_sun

for i, row in tqdm(df.iterrows(), total=len(df)):
    # If invalid scenario, skip
    if row['valid'] == False:
        continue
    
    # Initialize variables
    P1 = row['P1']; P2 = row['P2']
    j = row['j']
    PTTV = 1/np.abs(j/P2 - (j-1)/P1)
    
    e1 = row['e1']; w1 = row['w1']
    e2 = row['e2']; w2 = row['w2']
    
    mu1 = row['mu1']; mu2 = row['mu2']
    
    # Create TTVFast planets
    planet1 = models.Planet(
        mass=mu1,            # M_sun
        period=P1,           # days
        eccentricity=e1,
        inclination=90,      # degrees
        longnode=0,          # degrees
        argument=90+w1,      # degrees
        mean_anomaly=0       # degrees
    )
    planet2 = models.Planet(
        mass=mu2,
        period=P2,
        eccentricity=e2,
        inclination=90,
        longnode=0,
        argument=90+w2,
        mean_anomaly=0,
    )
    planets = [planet1, planet2]
    
    begin_time = -P2 * 200
    Time = begin_time   # days
    dt = P1 / 100       # days
    Total = max(250, PTTV * 2.5)  # days
    
    results = ttvfast.ttvfast(planets, stellar_mass, Time, dt, Total,
                              input_flag=1)

    # Create TTV signal from TTVFast Output
    out = pd.DataFrame(results['positions']).transpose()
    out.columns = ['planet', 'epoch', 'times', 'rsky', 'vsky']
    
    table0 = out[(out.planet == 0) & (out.times > 0)]
    n0 = table0.epoch
    tn0 = table0.times

    table1 = out[(out.planet == 1) & (out.times > 0)]
    n1 = table1.epoch
    tn1 = table1.times

    # If few TTVFast output, skip
    len_threshold = 10
    if (len(n0) <= len_threshold) or (len(n1) <= len_threshold):
        continue
        
    time0, ttv0, ttv0_err = ttvfit.return_ttv(epochs=n0, midtransits=tn0)
    time1, ttv1, ttv1_err = ttvfit.return_ttv(epochs=n1, midtransits=tn1)
    
    # Make sure ttv0 and ttv1 actually are centered around O-C = 0
    ttv0 = ttv0 - (np.max(ttv0) + np.min(ttv0)) / 2
    ttv1 = ttv1 - (np.max(ttv1) + np.min(ttv1)) / 2
    
    # Get sinusoidal TTV model parameters
    popt_ttv0, perr_ttv0, r2_0 = ttvfit.characterize_ttv(time=time0, ttv=ttv0, ttv_err=ttv0_err,
                                                         expected_period=PTTV, method='lmfit')
    popt_ttv1, perr_ttv1, r2_1 = ttvfit.characterize_ttv(time=time1, ttv=ttv1, ttv_err=ttv1_err,
                                                         expected_period=PTTV, method='lmfit')
    std0, std1 = np.std(ttv0), np.std(ttv1)

    # Write results to df_fit and df_ttv
    df_ttv.loc[i] = {
        "t0": time0,
        "ttv0": ttv0,
        "t1": time1,
        "ttv1": ttv1
    }
    df_fit.loc[i, df_fit_cols]  = [*popt_ttv0, std0, *popt_ttv1, std1, r2_0, r2_1]

100%|█████████████████████████████████████████████████████████████████████████| 100000/100000 [09:48<00:00, 169.81it/s]


In [5]:
# If output fit has no values, convert valid from TRUE to FALSE
output_has_nan = df_fit[df_fit_cols].isna().any(axis=1)
df_fit.loc[output_has_nan, 'valid'] = False

# If output has PTTV wildly different from each other, convert valid from TRUE to FALSE
Pttv_ratio_threshold = 1.05
Pttv_not_similar = (df_fit['Pttv1'] / df_fit['Pttv2'] > Pttv_ratio_threshold) | (df_fit['Pttv2'] / df_fit['Pttv1'] > Pttv_ratio_threshold)
df_fit.loc[Pttv_not_similar, 'valid'] = False

In [6]:
# Export dataframes
df_ttv.to_csv("deckagol_TTVFast_outputs.csv", index=False)
df_fit.to_csv("deckagol_TTVFast_fits.csv", index=False)